## Testes API SPTRANS

In [18]:
import requests
import pandas as pd
from minio import Minio
from minio.error import S3Error
import io
import json

In [12]:
minio_client = Minio(
    "localhost:9000",  # ou o IP ou hostname onde o MinIO está rodando
    access_key="uJ2LAihTN0YWgRmh5Wdd",
    secret_key="5XlZJa03fIrs1hkaB5v7QZEEPqAzFSpchMVSyobD",
    secure=False  # desativar SSL se estiver rodando localmente
)

#### API

In [2]:
class OlhoVivoAPI:
    def __init__(self, token):
        self.base_url = "http://api.olhovivo.sptrans.com.br/v2.1"
        self.token = token
        self.session = requests.Session()  # Sessão para manter o estado

    def autenticar(self):
        auth_url = f"{self.base_url}/Login/Autenticar?token={self.token}"
        # print(auth_url)
        response = self.session.post(auth_url)
        print(f"Autenticação: {response.text}")
        
    
    def buscar_linha(self, termos_busca):
        url = f"{self.base_url}/Linha/Buscar?termosBusca={termos_busca}"

        response = self.session.get(url)  # Usando a mesma sessão com cookie

        # Verifica se a requisição foi bem-sucedida
        if response.status_code == 200:
            return response.json()  # Retorna o JSON da resposta
        else:
            print(f"Erro na requisição: {response.status_code}, Resposta: {response.text}")
            return None
        
    def buscar_paradas(self, termos_busca):
        url = f"{self.base_url}/Parada/Buscar?termosBusca={termos_busca}"

        response = self.session.get(url) 

        if response.status_code == 200:
            return response.json()
        else:
            print(f"Erro na requisição: {response.status_code}, Resposta: {response.text}")
            return None
        
    def buscar_paradas_por_linha(self, codigo_linha):
        url = f"{self.base_url}/Parada/BuscarParadasPorLinha?codigoLinha={codigo_linha}"

        response = self.session.get(url)

        if response.status_code == 200:
            return response.json() 
        else:
            print(f"Erro na requisição: {response.status_code}, Resposta: {response.text}")
            return None
        
    def posicao_veiculos(self):
        url = f"{self.base_url}/Posicao"

        response = self.session.get(url)

        if response.status_code == 200:
            return response.json() 
        else:
            print(f"Erro na requisição: {response.status_code}, Resposta: {response.text}")
            return None

    def posicao_veiculos_codigo_linha(self, codigo_linha):
        url = f"{self.base_url}/Posicao/Linha?codigoLinha={codigo_linha}"

        response = self.session.get(url)

        if response.status_code == 200:
            return response.json() 
        else:
            print(f"Erro na requisição: {response.status_code}, Resposta: {response.text}")
            return None

#### Token

In [3]:
token = "ae2f510b572201ce998ea8e669d6c39066609ee0cad28f678d46ae5ac986306a"
api = OlhoVivoAPI(token)

In [4]:
api.autenticar()

Autenticação: true


In [5]:
# busca linhas 
linhas = api.buscar_linha("Cohab Brasilândia")
df_linhas = pd.DataFrame(linhas)

In [6]:
df_linhas

,cl,lc,lt,sl,tl,tp,ts
0,833,False,1021,1,10,TERM. PIRITUBA,COHAB BRASILÂNDIA
1,33601,False,1021,2,10,TERM. PIRITUBA,COHAB BRASILÂNDIA
2,2453,False,1036,1,10,CONEXÃO VL. IÓRIO,COHAB BRASILÂNDIA
3,35221,False,1036,2,10,CONEXÃO VL. IÓRIO,COHAB BRASILÂNDIA
4,877,False,9009,1,10,TERM. CACHOEIRINHA,COHAB BRASILÂNDIA
5,33645,False,9009,2,10,TERM. CACHOEIRINHA,COHAB BRASILÂNDIA
6,571,False,971C,1,10,METRÔ SANTANA,COHAB BRASILÂNDIA
7,33339,False,971C,2,10,METRÔ SANTANA,COHAB BRASILÂNDIA


In [33]:
paradas = api.buscar_paradas("PÇA")
df_paradas = pd.DataFrame(paradas)

In [34]:
df_paradas

,cp,np,ed,py,px
0,130001304,ENG. DIAS DE BARROS B/C,PÇA. ZUMBI DOS PALMARES,-23.479126,-46.673287
1,500001315,PARADA INACIO LEITAO B/C,PÇA. BENEDICTO DURVAL PESTANA BARBOSA,-23.486730,-46.682207


In [49]:
buscar_paradas = api.buscar_paradas_por_linha("34041")
df_buscar_paradas = pd.DataFrame(buscar_paradas)

In [50]:
df_buscar_paradas

,cp,np,ed,py,px
0,614432,BABY BARIONY C/B,R PEDRO MACHADO,-23.527608,-46.671511
1,7014416,ANGELICA C/B,R DR. ALBUQUERQUE LINS,-23.534377,-46.654322
2,60014604,SESC POMPEIA C/B,VDTO POMPEIA/ R JOAQUIM FERREIRA,-23.525262,-46.682254
3,60016783,PARADA PALMEIRAS C/B,R PADRE ANTONIO TOMAS/ AV POMPEIA,-23.525653,-46.679493
4,60016785,ANTARTICA C/B,R ENGENHEIRO STEVENSON/ PC SOUSA ARANHA,-23.526220,-46.674187
5,60016789,PQ. DA ÁGUA BRANCA C/B,R DONA GERMAINE BUCHARD/ R MINISTRO GODOI,-23.529636,-46.667777
6,480014608,TIBERIO C/B,R TIBERIO/ R MENFIS,-23.522875,-46.688219
7,480014609,DUILIO C/B,R DUILIO/ R CLAUDIO,-23.522627,-46.692121
8,480014610,TENDAL DA LAPA C/B,R ESPARTACO/ R MORAIS E SILVA,-23.522339,-46.696246
9,700006085,ROSA E SILVA C/B,R OLIMPIA DE ALMEIDA PRADO/ R LOPES DE OLIVEIRA,-23.532629,-46.657465


In [7]:
posicao = api.posicao_veiculos()
df_posicao = pd.DataFrame(posicao)

In [13]:
posicao

{'hr': '13:17',
 'l': [{'c': '6099-10',
   'cl': 34355,
   'sl': 2,
   'lt0': 'DIVISA DE EMBU-GUAÇU',
   'lt1': 'TERM. GRAJAÚ',
   'qv': 3,
   'vs': [{'p': 66151,
     'a': True,
     'ta': '2024-09-14T16:16:47Z',
     'py': -23.7954495,
     'px': -46.7301335,
     'sv': None,
     'is': None},
    {'p': 66198,
     'a': True,
     'ta': '2024-09-14T16:16:43Z',
     'py': -23.737344,
     'px': -46.697708500000005,
     'sv': None,
     'is': None},
    {'p': 66754,
     'a': True,
     'ta': '2024-09-14T16:16:57Z',
     'py': -23.7367155,
     'px': -46.697201,
     'sv': None,
     'is': None}]},
  {'c': '6970-10',
   'cl': 1505,
   'sl': 1,
   'lt0': 'TERM. STO. AMARO',
   'lt1': 'TERM. GRAJAÚ',
   'qv': 4,
   'vs': [{'p': 61071,
     'a': True,
     'ta': '2024-09-14T16:16:54Z',
     'py': -23.735712499999998,
     'px': -46.69611125,
     'sv': None,
     'is': None},
    {'p': 61661,
     'a': True,
     'ta': '2024-09-14T16:16:26Z',
     'py': -23.6967275,
     'px': -46.714889

In [14]:
bucket_name = "raw"

In [19]:
json_data = json.dumps(posicao)
json_bytes = io.BytesIO(json_data.encode("utf-8"))
minio_client.put_object(
    bucket_name,  # Nome do bucket
    "teste/data.json",  # Nome do arquivo que será salvo no MinIO
    json_bytes,   # O conteúdo do arquivo em bytes
    len(json_data.encode('utf-8'))  # Tamanho do arquivo em bytes
)

In [60]:
df_posicao

,hr,l
0,20:33,"{'c': '3686-10', 'cl': 166, 'sl': 1, 'lt0': 'T..."
1,20:33,"{'c': '8050-31', 'cl': 415, 'sl': 1, 'lt0': 'L..."
2,20:33,"{'c': '807J-10', 'cl': 34826, 'sl': 2, 'lt0': ..."
3,20:33,"{'c': '695T-10', 'cl': 1322, 'sl': 1, 'lt0': '..."
4,20:33,"{'c': '7053-10', 'cl': 1887, 'sl': 1, 'lt0': '..."
...,...,...
2020,20:33,"{'c': '8084-10', 'cl': 2607, 'sl': 1, 'lt0': '..."
2021,20:33,"{'c': '1730-10', 'cl': 33633, 'sl': 2, 'lt0': ..."
2022,20:33,"{'c': '129F-41', 'cl': 35073, 'sl': 2, 'lt0': ..."
2023,20:33,"{'c': '4085-10', 'cl': 2156, 'sl': 1, 'lt0': '..."


In [55]:
posicao_codigo = api.posicao_veiculos_codigo_linha(34041)
df_posicao_linha = pd.DataFrame(posicao_codigo)


In [56]:
df_posicao_linha

,hr,vs
0,20:30,"{'p': '11526', 'a': True, 'ta': '2024-09-12T23..."
1,20:30,"{'p': '11527', 'a': True, 'ta': '2024-09-12T23..."
